<a href="https://colab.research.google.com/github/t-mcneal/loanx/blob/master/LoanX.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LoanX

Welcome to LoanX!

LoanX helps student loan borrowers calculate and view their monthly payment breakdowns. The LoanX tool is designed on Colaboratory.



____

<p><img alt="Colaboratory logo" height="45px" src="/img/colab_favicon.ico" align="left" hspace="10px" vspace="0px"></p>

<h1>What is Colaboratory?</h1>

Colaboratory, or "Colab" for short, enables the writing and execution of the Python programming language in a browser. Created by Google, the Colab cloud platform can harness the full power of popular Python libraries for data analysis and visualization. The document you are reading is not a static web page, but an interactive environment called a Colab notebook.

*source: [Google Colab](https://colab.research.google.com/notebooks/intro.ipynb#scrollTo=GJBs_flRovLc)*


## **Getting Started** 

Colab uses cells like the one below to host Python code. To execute the code in the cells, select the cells with a click and then **press the play button** to the left of the cell. Also, it is important to execute the cells **in order** when interacting with Colab environments. Try out the cell below to practice using Colab cells. After clicking the play button, the cell should show the phrase, "Hello World!"




In [ ]:
#@title << Play
# double click the empty area on the right to hide code  -->



# Python code

def hello():
  return 'Hello World!'

hello = hello()
print(hello)


## **Next Steps**

You have now learned the basics of using Colab to operate the LoanX tool. Scroll down to start using LoanX.

___

# LoanX

____

**CALCULATE PAYMENTS** | Calculate the monthly payment amount needed to repay your loan on time

*   If using this monthly payment amount estimator for multiple loans, calculate each one separately and add up the payment estimates.
*   Enter interest rate as a decimal or percent value. LoanX will assume your interest rate is less than 100%.
*   Please note, the monthly payment amount is an estimate provided for information purposes only. *(1)*







In [ ]:
#@title << Calculate Monthly Payment
# double click the form on the right to hide code  -->

Loan_Amount =   0#@param {type:"number"}
Interest_Rate =   0#@param {type:"number"}
Years_to_Repay =   0#@param {type:"number"}


import numpy as np
import pandas as pd
import math
from google.colab import files


# FUNCTIONS

def numsValid(lst):
  """Return True if all numbers in the list are greater than 0.
  Returns False otherwise.

  KeyWord arguments:
  lst -- list of numbers
  """
  for nums in lst:
    if nums <= 0:
      return False
    else:
      try: 
        float(nums)
      except:
        return False
  return True


def monPayment(loan, intRate, years):
  """Return monthly payment amount.

  After calculating the monthly payment, the dollar amount can
  equal long double values (i.e. 105.12893837). This method
  rounds to two decimal places to present the number as a normal 
  dollar amount. The rounding needs to be very accurate, and 
  consider financial portions that are less than 1 cent. 
    
  Keyword arguments:
  loan -- laon amount
  intRate -- interest rate
  years -- years to repay loan
  """
  j = intRate / 12
  n = years * 12
  monPay = loan * (j / (1 - (1 + j)**-n))
  strMonPay = str(monPay) 
  if '.' in strMonPay:  
    strMonPay = strMonPay.split('.') # parse string to get first 2 numbers after decimal 
    r = strMonPay[1]
    if len(r) > 2:
      r = int(r[:2]) + 1 
    elif len(r) == 1:
      r = int(r + '0')
    else:
      r = int(r[:2])
    monPay = math.floor(monPay) + (r / 100)
  return f'${monPay:,.2f}'


def checkRate(intRate):
  """Check if interest rate is decimal or percentage.

  Interest rates can be entered on the LoanX forms as a 
  decimal or percent value. LoanX will assume the interest
  rate is less than 100%.

  Keyword arguments:
  intRate -- interest rate
  """
  if intRate >= 1:
    intRate = intRate / 100
  return intRate


def download(dataFrame):
  """Download Pandas dataframe as a CSV file.

  Keyword arguments:
  dataFrame -- Pandas data frame
  """
  df = dataFrame
  df.to_csv('amortization_schedule.csv') 
  files.download('amortization_schedule.csv')


class StudentLoan(object):

    def __init__(self, loan=1000, intRate=0.001,
                 payment=100, years=1):
        """The constructor.
        
        Keyword arguments:
        loan -- student loan amount (default 1000)
        intRate -- interest rate (default 0.001)
        payment -- monthly payment amount (default 100)
        years -- years to repay loan (default 1)
        """
        self.loan = loan
        self.intRate = intRate
        self.payment = payment
        self.years = years

    def setLoan(self, value):
        """Set the loan amount"""
        self.loan = value

    def setIntRate(self, value):
        """Set the interest rate"""
        self.intRate = value

    def setPayment(self, value):
        """Set the monthly payment amount"""
        self.payment = value

    def setYears(self, value):
        """Set the number of years to repay the loan"""
        self.years = value

    # List of variables names in the methods below:
    #
    # pb -- Principal Balance
    # intPaid -- Interest Paid
    # prinPaid -- Principal Paid
    # nb -- New Balance

    def repay(self):
        """Return time it will take to repay loan"""
        nb = self.loan
        lastMonth = self.years * 12
        for i in range(lastMonth):
            if nb > 0:
                month = i + 1
                pb = nb
                nb = self.__newBal(pb)
        if month == lastMonth and nb > 0:
            payIncr = self.__payIncrease()
            return payIncr
        else:
            payDet = self.__payDetails(month)
            return payDet
            

    # NumPy and Pandas is used in the schedule() method
    # below to create a data frame. The created data frame
    # is an amortization schedule. 
    
    def schedule(self):
        """Return loan amortization schedule"""
        pd.set_option('max_rows', 360)
        nb = self.loan
        payment = self.payment
        scheduleData = []
        lastMonth = self.years * 12
        for i in range(lastMonth):
            dataRow = []
            if nb <= 0:
                break
            else:
                month = i + 1
                pb = nb
                intPaid = self.intRate / 12 * pb
                prinPaid = self.payment - intPaid
                if prinPaid > pb:
                  prinPaid = pb
                  payment = intPaid + prinPaid
                nb = pb - prinPaid
                dataRow.append(month) 
                dataRow.append(f'${pb:,.2f}')
                dataRow.append(f'${payment:,.2f}')
                dataRow.append(f'${intPaid:,.2f}')
                dataRow.append(f'${prinPaid:,.2f}')
                if nb > 0:
                    dataRow.append(f'${nb:,.2f}')
                else:
                    dataRow.append('$0.00')
                scheduleData.append(dataRow)
        
        # np -- NumPy
        # pd -- Pandas
        userData = np.array(scheduleData)
        column_names = ['Month', 'Principal_Balance', 'Payment', 'Interest_Paid', 'Principal_Paid', 'New_Balance']
        df = pd.DataFrame(data=userData, columns=column_names)
        df.set_index('Month', inplace=True)
        return df

    # Private Method
    def __newBal(self, pb):
        """Return new principal balance of loan after making a payment.
        
        Keyword arguments:
        pb -- principal balance
        """
        intPaid = self.intRate / 12 * pb
        prinPaid = self.payment - intPaid
        nb = pb - prinPaid
        return nb

    # Private Method
    def __payDetails(self, month):
        """Return details of payment duration.
        
        Keyword arguements:
        month -- number of months it will take to repay loan
        """
        return f'''The ${self.loan:,.2f} loan will take {math.floor(month / 12)} years 
and {month % 12} months to repay with a monthly payment of ${self.payment:,.2f}.'''

    # Private Method
    def __payIncrease(self):
        """Return suggestion to increase monthly payment"""
        mPay = self.__monPayment(self.loan, self.intRate, self.years)
        word = 'year'
        if self.years > 1:
            word = word + 's'
        return f'''The ${self.loan:,.2f} loan will take over {self.years} years to repay
with a monthly payment of ${self.payment:,.2f}. \n\nIncrease monthly payment to ${mPay:,.2f} to repay the loan 
within {self.years} {word}.'''
        
    # Private Method
    def __monPayment(self, loan, intRate, years):
        """Return monthly payment amount.

        Keyword arguments:
        loan -- laon amount
        intRate -- interest rate
        years -- years to repay loan
        """
        j = intRate / 12
        n = years * 12
        monPay = loan * (j / (1 - (1 + j)**-n))
        strMonPay = str(monPay)
        if '.' in strMonPay:
            strMonPay = strMonPay.split('.')
            r = strMonPay[1]
            if len(r) > 2:
                r = int(r[:2]) + 1
            elif len(r) == 1:
                r = int(r + '0')
            else:
                r = int(r[:2])
            return math.floor(monPay) + (r / 100)
        return monPay

    def __repr__(self):
        """Representation of StudentLoan object"""
        print(f'StudentLoan({self.loan}, {self.intRate}, {self.payment}, {self.years})')

    def __str__(self):
        """String representation of StudentLoan object"""
        word = 'year'
        if self.years > 1:
            word = word + 's'
        print(f'''The ${self.loan:,.2f} student loan has an interest rate 
of {self.intRate * 100:.2f}% and monthly payment of ${self.payment:,.2f}.''')
        print()
        print(f'The loan must be repaid within {self.years} {word}.')





# Python code for "Calculate Monthly Payment" FORM    
try:
  valid = numsValid([Loan_Amount, Interest_Rate, Years_to_Repay])
  if valid:
    Interest_Rate = checkRate(Interest_Rate)
    payment = monPayment(Loan_Amount, Interest_Rate, Years_to_Repay)
    print(payment)
  else:
    print('All values must be a number greater than 0')
except:
  print('Sorry, we couldn\'t process your request.')
  print()
  print('''Please double check that you pressed play on all previous cells
  at least once.''')


______

**VIEW SCHEDULE** | Create an amortization schedule to see monthly payment breakdows

An **amortization schedule** shows monthly payment breakdowns of a loan. Create an amortization schedule below to view a detailed breakdown of your monthly payments.


**Explore:** If you're interested in exploring the possibility of repaying your loan faster, increase your monthly payment amount in the form below. By increasing your monthly payment amount, you can cut down on the duration of years and months it will take you to pay off the loan. Explore how you can reach your debt-free goals quicker!


In [ ]:
#@title << Create Amortization Schedule
# double click the form on the right to hide code  -->
#@markdown NOTE: You may have to scroll up a created schedule to view Month 1.

#@title Number fields
Loan_Amount =   0#@param {type:"number"}
Interest_Rate =   0#@param {type:"number"}
Years_to_Repay =   0#@param {type:"number"}
Monthly_Payment =   0#@param {type:"number"}


# Python code for "Create Amortization Schedule" FORM
try:
  valid = numsValid([Loan_Amount, Interest_Rate, Monthly_Payment, Years_to_Repay])
  if valid:
    Interest_Rate = checkRate(Interest_Rate)
    a = StudentLoan(Loan_Amount, Interest_Rate, Monthly_Payment, Years_to_Repay)
    details = a.repay()
    if "Increase" in details:
      print(details)
    else:
      print(details)
      print()
      display(a.schedule())
  else:
    print('All values must be a number greater than 0')
    print()
except:
  print('Sorry, we couldn\'t process your request.')
  print()
  print('''Please double check that you pressed play on all previous cells
  at least once.''')




_____

**DOWNLOAD FILE** | Download a CSV file of your student loan amortization schedule

In [ ]:
#@title << Download Amortization Schedule
# double click the empty area on the right to hide code  -->


# Python code for Downloading CSV file
try:
  download(a.schedule())
except:
  print('Sorry, we couldn\'t process your request.')
  print()
  print('''Please double check that you pressed play on all previous cells
  at least once.''')
  

____

## **DISCLAIMER** | Read the important disclaimer about LoanX

1. LoanX does not guarantee the estimator accuracy or applicability to a person's individual circumstances. The estimated monthly payment assumes the same payment amount and fixed interest rate for the life of the loan and does not account for a variable interest rate. The estimate does not account for missed payments, the use of deferment or forbearance, or any required minimum monthly payment amount for a particular loan. For new loans, this estimate does not account for any payments made during the in-school and separation or grace periods, or any interest that accrues or capitalizes during that time. For loans entering principal and interest repayment, this estimate does not factor in accruing interest or any payments made between now and when the loan enters principal and interest repayment.


© 2020 LoanX